In [1]:
from vpython import*
from queue import PriorityQueue

<IPython.core.display.Javascript object>

In [2]:
clock = 0

class Net:
    def __init__(self, driver, instance, time, on=0):
        self.value = None
        self.instance = instance
        self.driver = driver
        self.receivers = []
        self.on = on 
        self.time = time

    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
            
        for input in inputs:
            self.receivers.append(input)

    def set(self, value):
        if self.value == value:
            return 
        
        self.value = value
        
        if self.on:
            self.driver.evaluate()
            
        for c in self.receivers:
            c.set(value)
        
class Logic:
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        return

class Gate(Logic):        
    def __init__(self, name, instance, delay):
        Logic.__init__(self, name)
        self.instance = instance
        self.A = Net(self, 0, time=0, on=1)
        self.B = Net(self, 1, time=0, on=1)
        self.C = Net(self, 2, time=delay, on=1)
        
        
class Nand(Gate):       
    def __init__(self, name, instance):
        Gate.__init__(self, name, instance, 1)
        
    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(int(result))
        
class Nor(Gate):         
    def __init__(self, name, instance):
        Gate.__init__(self, name, instance, 1)

    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(int(result))
        
class Not(Logic):     
    def __init__(self, name, instance, delay):
        Logic.__init__(self, name)
        self.instance = instance
        self.A = Net(self, 0, time=0, on=1)
        self.B = Net(self, 2, time=delay)

    def evaluate(self):
        self.B.set(not self.A.value)
        
class Event:
    
    def __init__(self, net, t):
        self.instance = net 
        self.value = net.value 
        self.time = net.time + t
        self.type = net.instance
        
    def __lt__(self, other): return self.time < other.time
    
    def __le__(self, other): return self.time <= other.time
        
    def __eq__(self, other): return self.time == other.time
    
    def __gt__(self, other): return self.time > other.time
    
    def __ge__(self, other): return self.time >= other.time
        
    
        
def set_input(gate, a, b):
    gate.A.set(a)
    gate.B.set(b)
    
def update_net(gate, input_values, index):
    a,b = input_values[index]
    set_input(gate, a, b)
    
def get_event(eq1, eq2, eq3):
    e1 = eq1.get()
    e2 = eq2.get()
    e3 = eq3.get()
    
    return e1, e2, e3

In [3]:
t = 0
truth_values = [(0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]

event_q1 = PriorityQueue()
event_q2 = PriorityQueue()
event_q3 = PriorityQueue()

nand = Nand('nand', 0)

update_net(nand, truth_values, 0)

e1 = Event(nand.A, t)
e2 = Event(nand.B, t)
e3 = Event(nand.C, t - 1)

event_q1.put(e1)
event_q2.put(e2)
event_q3.put(e3)


# Verification of NAND nets (inputs and output)

In [3]:
scene2 = canvas()

<IPython.core.display.Javascript object>

In [3]:
g1 = graph(title="net1", xtitle='time', ytitle='value', fast=False, width=800)
g2 = graph(title="net2", xtitle='time', ytitle='value', fast=False, width=800)
g3 = graph(title="net3", xtitle='time', ytitle='value', fast=False, width=800)
net1 = gcurve(graph=g1, color=color.blue, width=4, markers=True, marker_color=color.black, label='net1')
net2 = gcurve(graph=g2, color=color.yellow, width=4, markers=True, marker_color=color.black, label='net2')
net3 = gcurve(graph=g3, color=color.red, width=4, markers=True, marker_color=color.black, label='nand1')

index = 1
while not (event_q1.empty() and event_q2.empty() and event_q3.empty()):
    # get next event
    event1, event2, event3 = get_event(event_q1, event_q2, event_q3)
    
    # update time
    t += event3.time - clock
    clock = t
    
    net1.plot(clock, event1.value)
    net2.plot(clock, event2.value)
    net3.plot(clock, event3.value)
    
    
    if index < len(truth_values):
        # update_net(event)
        update_net(nand, truth_values, index)
        index += 1
        
        next_event1 = Event(event1.instance, clock)
        next_event2 = Event(event2.instance, clock)
        next_event3 = Event(event3.instance, clock)
        
        # post new event
        event_q1.put(next_event1)
        event_q2.put(next_event2)
        event_q3.put(next_event3)

        # update visualization
        net1.plot(clock, next_event1.value)
        net2.plot(clock, next_event2.value)
        net3.plot(clock, next_event3.value)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NameError: name 'event_q1' is not defined

# NAND-Gate D Latch

In [4]:
clk = 0
sysClk = 0

clock_values = [0,1,0,1,0,1,0,1,0,1,0]
data_values = [1,1,1,0,0,0,0,0,0,0,0]

nand1 = Nand('nand1_S', 0)
nand2 = Nand('nand2_R', 1)
nand3 = Nand('nand3_Q', 2)
nand4 = Nand('nand4_Qb', 3)
not1 = Not('not1', 0, .5)

nand1.B.connect([nand2.A])
nand2.B.connect([not1.B])
not1.A.connect([nand1.A])
nand1.C.connect([nand3.A])
nand4.C.connect([nand3.B])
nand2.C.connect([nand4.B])
nand3.C.connect([nand4.A])

nand1.A.set(data_values[0])
nand1.B.set(clock_values[0])

eq_Q = PriorityQueue()
eq_Qb = PriorityQueue()
eq_D = PriorityQueue()
eq_clk = PriorityQueue()

ev_D = Event(nand1.A, clk)
ev_clk = Event(nand1.B, clk)
ev_Q = Event(nand4.A, clk)
ev_Qb = Event(nand3.B, clk)

eq_Q.put(ev_Q)
eq_Qb.put(ev_Qb)
eq_D.put(ev_D)
eq_clk.put(ev_clk)



0


In [5]:
scene3 = canvas()

<IPython.core.display.Javascript object>

In [6]:
gr1 = graph(title="net1", xtitle='time', ytitle='value', fast=False, width=800)
gr2 = graph(title="net2", xtitle='time', ytitle='value', fast=False, width=800)
gr3 = graph(title="net3", xtitle='time', ytitle='value', fast=False, width=800)
gr4 = graph(title="net3", xtitle='time', ytitle='value', fast=False, width=800)

Q_g = gcurve(graph=gr1, color=color.blue, width=4, markers=True, marker_color=color.black, label='Q')
Qb_g = gcurve(graph=gr2, color=color.yellow, width=4, markers=True, marker_color=color.black, label='Qb')
D_g = gcurve(graph=gr3, color=color.red, width=4, markers=True, marker_color=color.black, label='Data')
clk_g = gcurve(graph=gr4, color=color.red, width=4, markers=True, marker_color=color.black, label='Clk')

i = 1
while not eq_Q.empty():
    # get next event
    e1 = eq_Q.get()
    e2 = eq_Qb.get()
    e3 = eq_D.get()
    e4 = eq_clk.get()
    
    # update time
    clk += 1 - sysClk
    sysClk = clk
    
    Q_g.plot(sysClk, e1.value)
    Qb_g.plot(sysClk, e2.value)
    D_g.plot(sysClk, e3.value)
    clk_g.plot(sysClk, e4.value)
    
    if i < len(clock_values):
        # update_net(event)
        nand1.A.set(data_values[i])
        nand1.B.set(clock_values[i])
        i += 1
        
        next_e1 = Event(e1.instance, sysClk)
        next_e2 = Event(e2.instance, sysClk)
        next_e3 = Event(e3.instance, sysClk)
        next_e4 = Event(e4.instance, sysClk)
        
        # post new event
        eq_Q.put(next_e1)
        eq_Qb.put(next_e2)
        eq_D.put(next_e3)
        eq_clk.put(next_e4)
        
        # update visualization
        Q_g.plot(sysClk, next_e1.value)
        Qb_g.plot(sysClk, next_e2.value)
        D_g.plot(sysClk, next_e3.value)
        clk_g.plot(sysClk, next_e4.value)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>